#Détection de la langue d'un texte (projet 1)

Professeur: Alexis Nasr

Master 1

Aix-Marseille Université

**Le but de ce projet est de programmer un classifieur permettant de détecter la langue d'un texte à partir de la fréquence des bigrammes de ce dernier.**

## Préparation des données

On récupère les corpus de d'apprentissage et de test

In [ ]:
%%bash
rm train.tgz
wget http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/train.tgz
tar xvfz train.tgz


train/
train/uk_iu-ud-train.txt
train/zh_gsd-ud-train.txt
train/la_ittb-ud-train.txt
train/af_afribooms-ud-train.txt
train/be_hse-ud-train.txt
train/cs_cac-ud-train.txt
train/fro_srcmf-ud-train.txt
train/hsb_ufal-ud-train.txt
train/mt_mudt-ud-train.txt
train/mr_ufal-ud-train.txt
train/en_lines-ud-train.txt
train/fr_sequoia-ud-train.txt
train/cu_proiel-ud-train.txt
train/fi_ftb-ud-train.txt
train/ro_rrt-ud-train.txt
train/sv_talbanken-ud-train.txt
train/hi_hdtb-ud-train.txt
train/en_esl-ud-train.txt
train/pl_sz-ud-train.txt
train/cs_pdt-ud-train.txt
train/bg_btb-ud-train.txt
train/el_gdt-ud-train.txt
train/fr_partut-ud-train.txt
train/sme_giella-ud-train.txt
train/da_ddt-ud-train.txt
train/qhe_hiencs-ud-train.txt
train/lv_lvtb-ud-train.txt
train/no_nynorsk-ud-train.txt
train/cs_fictree-ud-train.txt
train/en_ewt-ud-train.txt
train/ga_idt-ud-train.txt
train/fr_spoken-ud-train.txt
train/ca_ancora-ud-train.txt
train/sl_sst-ud-train.txt
train/ur_udtb-ud-train.txt
train/got_proiel-ud-train.tx

rm: cannot remove 'train.tgz': No such file or directory
--2024-03-01 22:13:23--  http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/train.tgz
Resolving pageperso.lif.univ-mrs.fr (pageperso.lif.univ-mrs.fr)... 139.124.22.27
Connecting to pageperso.lif.univ-mrs.fr (pageperso.lif.univ-mrs.fr)|139.124.22.27|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pageperso.lis-lab.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/train.tgz [following]
--2024-03-01 22:13:24--  https://pageperso.lis-lab.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/train.tgz
Resolving pageperso.lis-lab.fr (pageperso.lis-lab.fr)... 139.124.22.27
Connecting to pageperso.lis-lab.fr (pageperso.lis-lab.fr)|139.124.22.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29281190 (28M) [application/x-gzip]
Saving to: ‘train.tgz’

     0K .......... .......... .......... .......... ..........  0%  220K 2m10s
    

**On fait la liste des fichiers dont on veut se servir pour construire les données d'apprentissage.**
**A chaque corpus on associe un identificateur de la langue (en, fr, it ...)**

In [ ]:
l_corpus_train=[
['en', './train/en_partut-ud-train.txt'],
['fr', './train/fr_sequoia-ud-train.txt'],
['it', './train/it_partut-ud-train.txt'],
['nl', './train/nl_lassysmall-ud-train.txt'],
['sl', './train/sl_sst-ud-train.txt'],
['es', './train/es_ancora-ud-train.txt'],
['pt', './train/pt_bosque-ud-train.txt'],
['de', './train/de_gsd-ud-train.txt'],
['ca', './train/ca_ancora-ud-train.txt']
]

**On construit un dictionnaire qui associe à chaque langue un identifiant numérique**

In [ ]:
def calculeCodeLangues(l_corpus):
    nbLangues = 0
    codeLangue = { }

    for corpus in l_corpus:
        idLangue = corpus[0]
        fichierCorpus = corpus[1]
        if not idLangue in codeLangue :
          print('langue :', idLangue, 'code = ', nbLangues)
          codeLangue[idLangue] = nbLangues
          nbLangues += 1
    return codeLangue

codeLangues = calculeCodeLangues(l_corpus_train)


langue : en code =  0
langue : fr code =  1
langue : it code =  2
langue : nl code =  3
langue : sl code =  4
langue : es code =  5
langue : pt code =  6
langue : de code =  7
langue : ca code =  8


**On extrait des corpus d'apprentissage des bigrammes dont on calcule la fréquence.**
**Les fréquences de bigrammes sont stockées dans un fichier dont le format est le suivant :**
**chaque ligne se présente sous la forme de l'identificateur d'une langue, suivie de la fréquence des différents bigrammes dans un ordre fixé (voir la fonction bigram_code())**

In [ ]:
import random
import sys

def calculFreq(bigrammes):
    somme = 0
    for elt in bigrammes :
        somme += elt
    i = 0
    while i < len(bigrammes):
        bigrammes[i] /= somme
        i = i + 1

def afficheBigrammes(bigrammes, fic):
    for frequence in bigrammes :
        print(frequence, ' ', file = fic, end='')
    print('', file = fic)

def char_code(c):
    if 'a' <= c and c <= 'z':
        return ord(c) - ord('a')
    elif c == ' ':
        return 26
    else :
        return 27


def bigram_code(c1, c2):
    return 28 * char_code(c1) + char_code(c2)


def process_corpus_random(corpus, maxBigrammes, maxTirage):
    try:
        fic = open(corpus, 'r')
    except IOError:
        print("le fichier", corpus, "n'existe pas")
        return None

    corpus_str = fic.read()
    longueur_corpus = len(corpus_str)
#    print("longueur corpus = ", longueur_corpus)
    fic.close()

    l_bigrammes = []
    tirage = 0
    while tirage < maxTirage:
        bigrammes = [0] * 784
        nbBigrammes = 0
        while nbBigrammes <= maxBigrammes :
            position = random.randint(0,longueur_corpus - 2)
            code_bigramme = bigram_code(corpus_str[position], corpus_str[position + 1])
            bigrammes[code_bigramme] += 1
            nbBigrammes += 1
        calculFreq(bigrammes)
        l_bigrammes.append(bigrammes)
        tirage += 1
    return l_bigrammes

def extract_bigrams(l_corpus, nbBigrammes, nbTirages, fichierSortie) :
    try:
        ficOut = open(fichierSortie, 'w')
    except IOError:
        print("le fichier", fichierSortie, "n'existe pas")
        exit

    for corpus in l_corpus:
        idLangue = corpus[0]
        fichierCorpus = corpus[1]
        print('traite corpus', fichierCorpus)
        l_bigrammes = process_corpus_random(fichierCorpus, nbBigrammes, nbTirages)
        for bigrammes in l_bigrammes:
            print(idLangue, ' ', file = ficOut, end='')
            afficheBigrammes(bigrammes, ficOut)
    ficOut.close()

extract_bigrams(l_corpus_train, 100, 100, 'train.dat')



traite corpus ./train/en_partut-ud-train.txt
traite corpus ./train/fr_sequoia-ud-train.txt
traite corpus ./train/it_partut-ud-train.txt
traite corpus ./train/nl_lassysmall-ud-train.txt
traite corpus ./train/sl_sst-ud-train.txt
traite corpus ./train/es_ancora-ud-train.txt
traite corpus ./train/pt_bosque-ud-train.txt
traite corpus ./train/de_gsd-ud-train.txt
traite corpus ./train/ca_ancora-ud-train.txt


**On met en forme les données de manière à pouvoir les fournir au réseau de neurones pour l'apprentissage**

In [ ]:
import numpy as np
import sys

def lectureDonnees(nomFichier, codeLangue):
    try:
        fic = open(nomFichier, 'r')
    except IOError:
        print("le fichier", nomFichier, "n'existe pas")
        return None

    nbLangues = len(codeLangue.keys())
    lx = []
    langues = []
    for ligne in fic:
        ligne = ligne.strip('\n\r')
        liste = ligne.split()
        langue = liste.pop(0)
        langues.append(codeLangue[langue])
        resultat = [float(x) for x in liste]
        lx.append(resultat)
    fic.close()

    ly = []
    for i in range(len(langues)):
        v = [0] * nbLangues
        v[langues[i]] = 1
        ly.append(v)

    import random
    t = list(zip(lx, ly))
    random.shuffle(t)
    lx, ly = list(zip(*t))

    x_train = np.array(lx, dtype="float")
    y_train = np.array(ly, dtype="int")
    print(len(x_train), "exemples lus")
    #print(x_train)
    return (x_train, y_train)



(x_train, y_train) = lectureDonnees('train.dat', codeLangues)


900 exemples lus


**On construit la structure du réseau et on fait l'apprentissage**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential()
nbLangues = len(codeLangues.keys())

print('nbLangues =', nbLangues)

model.add(Dense(units=100, activation='tanh', input_dim=28*28))
#model.add(Dropout(0.5))
model.add(Dense(units=nbLangues, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# x_train and y_train are Numpy arrays --just like in the Scikit-Learn API.
model.fit(x_train, y_train, epochs=20, batch_size=16, validation_split=0.2)



nbLangues = 9
Epoch 1/20
45/45 [==============================] - 1s 8ms/step - loss: 2.1742 - accuracy: 0.2583 - val_loss: 2.1495 - val_accuracy: 0.5111
Epoch 2/20
45/45 [==============================] - 0s 4ms/step - loss: 2.0998 - accuracy: 0.6056 - val_loss: 2.0627 - val_accuracy: 0.8111
Epoch 3/20
45/45 [==============================] - 0s 4ms/step - loss: 1.9781 - accuracy: 0.8417 - val_loss: 1.9267 - val_accuracy: 0.7611
Epoch 4/20
45/45 [==============================] - 0s 4ms/step - loss: 1.8030 - accuracy: 0.8833 - val_loss: 1.7453 - val_accuracy: 0.8333
Epoch 5/20
45/45 [==============================] - 0s 4ms/step - loss: 1.5927 - accuracy: 0.8792 - val_loss: 1.5431 - val_accuracy: 0.8389
Epoch 6/20
45/45 [==============================] - 0s 4ms/step - loss: 1.3810 - accuracy: 0.9042 - val_loss: 1.3432 - val_accuracy: 0.8556
Epoch 7/20
45/45 [==============================] - 0s 4ms/step - loss: 1.1826 - accuracy: 0.9056 - val_loss: 1.1709 - val_accuracy: 0.8556
Epoch 

**On charge les données de test**

In [ ]:
%%bash
rm test.tgz
wget http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/test.tgz
tar xvfz test.tgz


test/
test/cs_fictree-ud-test.txt
test/akk_pisandub-ud-test.txt
test/la_ittb-ud-test.txt
test/gl_ctg-ud-test.txt
test/bg_btb-ud-test.txt
test/sv_talbanken-ud-test.txt
test/hi_hdtb-ud-test.txt
test/fi_pud-ud-test.txt
test/et_edt-ud-test.txt
test/ga_idt-ud-test.txt
test/da_ddt-ud-test.txt
test/br_keb-ud-test.txt
test/ja_pud-ud-test.txt
test/cs_pdt-ud-test.txt
test/pl_lfg-ud-test.txt
test/no_nynorsk-ud-test.txt
test/pt_bosque-ud-test.txt
test/id_pud-ud-test.txt
test/bm_crb-ud-test.txt
test/kpv_ikdp-ud-test.txt
test/ar_padt-ud-test.txt
test/kk_ktb-ud-test.txt
test/ar_nyuad-ud-test.txt
test/hy_armtdp-ud-test.txt
test/fi_ftb-ud-test.txt
test/it_postwita-ud-test.txt
test/fa_seraji-ud-test.txt
test/en_partut-ud-test.txt
test/zh_pud-ud-test.txt
test/cs_pud-ud-test.txt
test/sk_snk-ud-test.txt
test/tr_imst-ud-test.txt
test/eu_bdt-ud-test.txt
test/got_proiel-ud-test.txt
test/ug_udt-ud-test.txt
test/kpv_lattice-ud-test.txt
test/ru_taiga-ud-test.txt
test/ru_syntagrus-ud-test.txt
test/wbp_ufal-ud-tes

rm: cannot remove 'test.tgz': No such file or directory
--2024-03-01 22:15:24--  http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/test.tgz
Resolving pageperso.lif.univ-mrs.fr (pageperso.lif.univ-mrs.fr)... 139.124.22.27
Connecting to pageperso.lif.univ-mrs.fr (pageperso.lif.univ-mrs.fr)|139.124.22.27|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pageperso.lis-lab.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/test.tgz [following]
--2024-03-01 22:15:24--  https://pageperso.lis-lab.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/test.tgz
Resolving pageperso.lis-lab.fr (pageperso.lis-lab.fr)... 139.124.22.27
Connecting to pageperso.lis-lab.fr (pageperso.lis-lab.fr)|139.124.22.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5342409 (5.1M) [application/x-gzip]
Saving to: ‘test.tgz’

     0K .......... .......... .......... .......... ..........  0%  220K 23s
    50K ...

In [ ]:
l_corpus_test=[
['en', './test/en_partut-ud-test.txt'],
['fr', './test/fr_sequoia-ud-test.txt'],
['it', './test/it_partut-ud-test.txt'],
['nl', './test/nl_lassysmall-ud-test.txt'],
['sl', './test/sl_sst-ud-test.txt'],
['es', './test/es_ancora-ud-test.txt'],
['pt', './test/pt_bosque-ud-test.txt'],
['de', './test/de_gsd-ud-test.txt'],
['ca', './test/ca_ancora-ud-test.txt']
]

In [ ]:
extract_bigrams(l_corpus_test, 500, 10, 'test.dat')
(x_test, y_test) = lectureDonnees('test.dat', codeLangues)

traite corpus ./test/en_partut-ud-test.txt
traite corpus ./test/fr_sequoia-ud-test.txt
traite corpus ./test/it_partut-ud-test.txt
traite corpus ./test/nl_lassysmall-ud-test.txt
traite corpus ./test/sl_sst-ud-test.txt
traite corpus ./test/es_ancora-ud-test.txt
traite corpus ./test/pt_bosque-ud-test.txt
traite corpus ./test/de_gsd-ud-test.txt
traite corpus ./test/ca_ancora-ud-test.txt
90 exemples lus


**On évalue le modèle sur les données de test**

In [ ]:
l_id = [''] * len(codeLangues)
for (key, val) in (codeLangues.items()):
  #print("key =", key , 'val =', val)
  l_id[int(val)] = key

In [ ]:

def argmax(l):
  i = 1
  max = l[0]
  arg = 0
  while i < len(l) :
    if l[i] > max :
      max = l[i]
      arg = i
    i = i + 1
  return arg

score = model.evaluate(x_test, y_test)
print('test.dat score = ', score)
l_pred = model.predict(x_test, batch_size=None, verbose=1, steps=None)
i = 0
while i < len(l_pred) :
  #print(l_pred[i])
  predicted = l_id[argmax(l_pred[i])]
  gold = l_id[argmax(y_test[i])]
  print('pred =', predicted, 'gold =', gold)
  i = i + 1


3/3 [==============================] - 0s 5ms/step - loss: 0.1960 - accuracy: 1.0000
test.dat score =  [0.19596706330776215, 1.0]
3/3 [==============================] - 0s 4ms/step
pred = en gold = en
pred = fr gold = fr
pred = ca gold = ca
pred = de gold = de
pred = sl gold = sl
pred = sl gold = sl
pred = fr gold = fr
pred = en gold = en
pred = sl gold = sl
pred = fr gold = fr
pred = ca gold = ca
pred = nl gold = nl
pred = it gold = it
pred = it gold = it
pred = en gold = en
pred = it gold = it
pred = de gold = de
pred = de gold = de
pred = en gold = en
pred = de gold = de
pred = nl gold = nl
pred = es gold = es
pred = sl gold = sl
pred = ca gold = ca
pred = fr gold = fr
pred = pt gold = pt
pred = sl gold = sl
pred = nl gold = nl
pred = pt gold = pt
pred = de gold = de
pred = de gold = de
pred = pt gold = pt
pred = es gold = es
pred = nl gold = nl
pred = it gold = it
pred = ca gold = ca
pred = en gold = en
pred = en gold = en
pred = pt gold = pt
pred = nl gold = nl
pred = it gold = it

#Code pour les **trigrammes**

Chargement des données d'entrainement

In [ ]:
%%bash
rm train.tgz
wget http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/train.tgz
tar xvfz train.tgz


train/
train/uk_iu-ud-train.txt
train/zh_gsd-ud-train.txt
train/la_ittb-ud-train.txt
train/af_afribooms-ud-train.txt
train/be_hse-ud-train.txt
train/cs_cac-ud-train.txt
train/fro_srcmf-ud-train.txt
train/hsb_ufal-ud-train.txt
train/mt_mudt-ud-train.txt
train/mr_ufal-ud-train.txt
train/en_lines-ud-train.txt
train/fr_sequoia-ud-train.txt
train/cu_proiel-ud-train.txt
train/fi_ftb-ud-train.txt
train/ro_rrt-ud-train.txt
train/sv_talbanken-ud-train.txt
train/hi_hdtb-ud-train.txt
train/en_esl-ud-train.txt
train/pl_sz-ud-train.txt
train/cs_pdt-ud-train.txt
train/bg_btb-ud-train.txt
train/el_gdt-ud-train.txt
train/fr_partut-ud-train.txt
train/sme_giella-ud-train.txt
train/da_ddt-ud-train.txt
train/qhe_hiencs-ud-train.txt
train/lv_lvtb-ud-train.txt
train/no_nynorsk-ud-train.txt
train/cs_fictree-ud-train.txt
train/en_ewt-ud-train.txt
train/ga_idt-ud-train.txt
train/fr_spoken-ud-train.txt
train/ca_ancora-ud-train.txt
train/sl_sst-ud-train.txt
train/ur_udtb-ud-train.txt
train/got_proiel-ud-train.tx

--2024-03-01 22:15:45--  http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/train.tgz
Resolving pageperso.lif.univ-mrs.fr (pageperso.lif.univ-mrs.fr)... 139.124.22.27
Connecting to pageperso.lif.univ-mrs.fr (pageperso.lif.univ-mrs.fr)|139.124.22.27|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pageperso.lis-lab.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/train.tgz [following]
--2024-03-01 22:15:45--  https://pageperso.lis-lab.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/train.tgz
Resolving pageperso.lis-lab.fr (pageperso.lis-lab.fr)... 139.124.22.27
Connecting to pageperso.lis-lab.fr (pageperso.lis-lab.fr)|139.124.22.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29281190 (28M) [application/x-gzip]
Saving to: ‘train.tgz’

     0K .......... .......... .......... .......... ..........  0%  220K 2m10s
    50K .......... .......... .......... .......... .........

Définition des corpus

In [ ]:
l_corpus_train=[
['en', './train/en_partut-ud-train.txt'],
['fr', './train/fr_sequoia-ud-train.txt'],
['it', './train/it_partut-ud-train.txt'],
['nl', './train/nl_lassysmall-ud-train.txt'],
['sl', './train/sl_sst-ud-train.txt'],
['es', './train/es_ancora-ud-train.txt'],
['pt', './train/pt_bosque-ud-train.txt'],
['de', './train/de_gsd-ud-train.txt'],
['ca', './train/ca_ancora-ud-train.txt']
]

Association pour chaque langue d'un code numérique

In [ ]:
def calculeCodeLangues(l_corpus):
    nbLangues = 0
    codeLangue = { }

    for corpus in l_corpus:
        idLangue = corpus[0]
        fichierCorpus = corpus[1]
        if not idLangue in codeLangue :
          print('langue :', idLangue, 'code = ', nbLangues)
          codeLangue[idLangue] = nbLangues
          nbLangues += 1
    return codeLangue

codeLangues = calculeCodeLangues(l_corpus_train)

langue : en code =  0
langue : fr code =  1
langue : it code =  2
langue : nl code =  3
langue : sl code =  4
langue : es code =  5
langue : pt code =  6
langue : de code =  7
langue : ca code =  8


Extraction des corpus d'apprentissage des trigrammes dont on calcule la fréquence

In [ ]:
import random
import sys

def calculFreq(trigrammes):
    somme = 0
    for elt in trigrammes :
        somme += elt
    i = 0
    while i < len(trigrammes):
        trigrammes[i] /= somme
        i = i + 1

def afficheTrigrammes(trigrammes, fic):
    for frequence in trigrammes :
        print(frequence, ' ', file = fic, end='')
    print('', file = fic)

def char_code(c):
    if 'a' <= c and c <= 'z':
        return ord(c) - ord('a')
    elif c == ' ':
        return 26
    else :
        return 27

def trigram_code(c1, c2, c3):
  return 28^2 * char_code(c1) + 28*char_code(c2) + char_code(c3)


def process_corpus_random(corpus, maxTrigrammes, maxTirage):
    try:
        fic = open(corpus, 'r')
    except IOError:
        print("le fichier", corpus, "n'existe pas")
        return None

    corpus_str = fic.read()
    longueur_corpus = len(corpus_str)
#    print("longueur corpus = ", longueur_corpus)
    fic.close()

    l_trigrammes = []
    tirage = 0
    while tirage < maxTirage:
        trigrammes = [0] * 21952
        nbTrigrammes = 0
        while nbTrigrammes <= maxTrigrammes :
            position = random.randint(0,longueur_corpus - 3) #-3
            code_trigramme = trigram_code(corpus_str[position], corpus_str[position + 1], corpus_str[position + 2])
            trigrammes[code_trigramme] += 1
            nbTrigrammes += 1
        calculFreq(trigrammes)
        l_trigrammes.append(trigrammes)
        tirage += 1
    return l_trigrammes


def extract_trigrams(l_corpus, nbTrigrammes, nbTirages, fichierSortie) :
    try:
        ficOut = open(fichierSortie, 'w')
    except IOError:
        print("le fichier", fichierSortie, "n'existe pas")
        exit

    for corpus in l_corpus:
        idLangue = corpus[0]
        fichierCorpus = corpus[1]
        print('traite corpus', fichierCorpus)
        l_trigrammes = process_corpus_random(fichierCorpus, nbTrigrammes, nbTirages)
        for trigrammes in l_trigrammes:
            print(idLangue, ' ', file = ficOut, end='')
            afficheTrigrammes(trigrammes, ficOut)
    ficOut.close()

extract_trigrams(l_corpus_train, 100, 100, 'train.dat')

traite corpus ./train/en_partut-ud-train.txt
traite corpus ./train/fr_sequoia-ud-train.txt
traite corpus ./train/it_partut-ud-train.txt
traite corpus ./train/nl_lassysmall-ud-train.txt
traite corpus ./train/sl_sst-ud-train.txt
traite corpus ./train/es_ancora-ud-train.txt
traite corpus ./train/pt_bosque-ud-train.txt
traite corpus ./train/de_gsd-ud-train.txt
traite corpus ./train/ca_ancora-ud-train.txt


 Mise en forme des données de manière à pouvoir les fournir au réseau de neurones pour l'apprentissage

In [ ]:
import numpy as np
import sys

def lectureDonnees(nomFichier, codeLangue):
    try:
        fic = open(nomFichier, 'r')
    except IOError:
        print("le fichier", nomFichier, "n'existe pas")
        return None

    nbLangues = len(codeLangue.keys())
    lx = []
    langues = []
    for ligne in fic:
        ligne = ligne.strip('\n\r')
        liste = ligne.split()
        langue = liste.pop(0)
        langues.append(codeLangue[langue])
        resultat = [float(x) for x in liste]
        lx.append(resultat)
    fic.close()

    ly = []
    for i in range(len(langues)):
        v = [0] * nbLangues
        v[langues[i]] = 1
        ly.append(v)

    import random
    t = list(zip(lx, ly))
    random.shuffle(t)
    lx, ly = list(zip(*t))

    x_train = np.array(lx, dtype="float")
    y_train = np.array(ly, dtype="int")
    print(len(x_train), "exemples lus")
    #print(x_train)
    return (x_train, y_train)



(x_train, y_train) = lectureDonnees('train.dat', codeLangues)


900 exemples lus


 Construction de la structure du réseau et réalisation de l'apprentissage

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential()
nbLangues = len(codeLangues.keys())

print('nbLangues =', nbLangues)

model.add(Dense(units=100, activation='tanh', input_dim=28*28*28))
#model.add(Dropout(0.5))
model.add(Dense(units=nbLangues, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# x_train and y_train are Numpy arrays --just like in the Scikit-Learn API.
model.fit(x_train, y_train, epochs=20, batch_size=16, validation_split=0.2)

nbLangues = 9
Epoch 1/20
45/45 [==============================] - 2s 30ms/step - loss: 2.1779 - accuracy: 0.3125 - val_loss: 2.1596 - val_accuracy: 0.4111
Epoch 2/20
45/45 [==============================] - 1s 25ms/step - loss: 2.1069 - accuracy: 0.5972 - val_loss: 2.0876 - val_accuracy: 0.5222
Epoch 3/20
45/45 [==============================] - 2s 37ms/step - loss: 1.9941 - accuracy: 0.6917 - val_loss: 1.9703 - val_accuracy: 0.7056
Epoch 4/20
45/45 [==============================] - 2s 42ms/step - loss: 1.8348 - accuracy: 0.8528 - val_loss: 1.8135 - val_accuracy: 0.7611
Epoch 5/20
45/45 [==============================] - 2s 36ms/step - loss: 1.6415 - accuracy: 0.9444 - val_loss: 1.6391 - val_accuracy: 0.8722
Epoch 6/20
45/45 [==============================] - 1s 24ms/step - loss: 1.4358 - accuracy: 0.9778 - val_loss: 1.4505 - val_accuracy: 0.9111
Epoch 7/20
45/45 [==============================] - 1s 25ms/step - loss: 1.2311 - accuracy: 0.9861 - val_loss: 1.2845 - val_accuracy: 0.9611

**On charge les données de test**

In [ ]:
%%bash
rm test.tgz
wget http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/test.tgz
tar xvfz test.tgz


test/
test/cs_fictree-ud-test.txt
test/akk_pisandub-ud-test.txt
test/la_ittb-ud-test.txt
test/gl_ctg-ud-test.txt
test/bg_btb-ud-test.txt
test/sv_talbanken-ud-test.txt
test/hi_hdtb-ud-test.txt
test/fi_pud-ud-test.txt
test/et_edt-ud-test.txt
test/ga_idt-ud-test.txt
test/da_ddt-ud-test.txt
test/br_keb-ud-test.txt
test/ja_pud-ud-test.txt
test/cs_pdt-ud-test.txt
test/pl_lfg-ud-test.txt
test/no_nynorsk-ud-test.txt
test/pt_bosque-ud-test.txt
test/id_pud-ud-test.txt
test/bm_crb-ud-test.txt
test/kpv_ikdp-ud-test.txt
test/ar_padt-ud-test.txt
test/kk_ktb-ud-test.txt
test/ar_nyuad-ud-test.txt
test/hy_armtdp-ud-test.txt
test/fi_ftb-ud-test.txt
test/it_postwita-ud-test.txt
test/fa_seraji-ud-test.txt
test/en_partut-ud-test.txt
test/zh_pud-ud-test.txt
test/cs_pud-ud-test.txt
test/sk_snk-ud-test.txt
test/tr_imst-ud-test.txt
test/eu_bdt-ud-test.txt
test/got_proiel-ud-test.txt
test/ug_udt-ud-test.txt
test/kpv_lattice-ud-test.txt
test/ru_taiga-ud-test.txt
test/ru_syntagrus-ud-test.txt
test/wbp_ufal-ud-tes

--2024-03-01 22:17:29--  http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/test.tgz
Resolving pageperso.lif.univ-mrs.fr (pageperso.lif.univ-mrs.fr)... 139.124.22.27
Connecting to pageperso.lif.univ-mrs.fr (pageperso.lif.univ-mrs.fr)|139.124.22.27|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pageperso.lis-lab.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/test.tgz [following]
--2024-03-01 22:17:31--  https://pageperso.lis-lab.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/test.tgz
Resolving pageperso.lis-lab.fr (pageperso.lis-lab.fr)... 139.124.22.27
Connecting to pageperso.lis-lab.fr (pageperso.lis-lab.fr)|139.124.22.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5342409 (5.1M) [application/x-gzip]
Saving to: ‘test.tgz’

     0K .......... .......... .......... .......... ..........  0%  219K 24s
    50K .......... .......... .......... .......... ..........  1% 

In [ ]:
l_corpus_test=[
['en', './test/en_partut-ud-test.txt'],
['fr', './test/fr_sequoia-ud-test.txt'],
['it', './test/it_partut-ud-test.txt'],
['nl', './test/nl_lassysmall-ud-test.txt'],
['sl', './test/sl_sst-ud-test.txt'],
['es', './test/es_ancora-ud-test.txt'],
['pt', './test/pt_bosque-ud-test.txt'],
['de', './test/de_gsd-ud-test.txt'],
['ca', './test/ca_ancora-ud-test.txt']
]

In [ ]:
extract_trigrams(l_corpus_test, 500, 10, 'test.dat')
(x_test, y_test) = lectureDonnees('test.dat', codeLangues)

traite corpus ./test/en_partut-ud-test.txt
traite corpus ./test/fr_sequoia-ud-test.txt
traite corpus ./test/it_partut-ud-test.txt
traite corpus ./test/nl_lassysmall-ud-test.txt
traite corpus ./test/sl_sst-ud-test.txt
traite corpus ./test/es_ancora-ud-test.txt
traite corpus ./test/pt_bosque-ud-test.txt
traite corpus ./test/de_gsd-ud-test.txt
traite corpus ./test/ca_ancora-ud-test.txt
90 exemples lus


**On évalue le modèle sur les données de test**

In [ ]:
l_id = [''] * len(codeLangues)
for (key, val) in (codeLangues.items()):
  #print("key =", key , 'val =', val)
  l_id[int(val)] = key

In [ ]:

def argmax(l):
  i = 1
  max = l[0]
  arg = 0
  while i < len(l) :
    if l[i] > max :
      max = l[i]
      arg = i
    i = i + 1
  return arg

score = model.evaluate(x_test, y_test)
print('test.dat score = ', score)
l_pred = model.predict(x_test, batch_size=None, verbose=1, steps=None)
i = 0
while i < len(l_pred) :
  #print(l_pred[i])
  predicted = l_id[argmax(l_pred[i])]
  gold = l_id[argmax(y_test[i])]
  print('pred =', predicted, 'gold =', gold)
  i = i + 1


3/3 [==============================] - 0s 11ms/step - loss: 0.2108 - accuracy: 1.0000
test.dat score =  [0.2108108103275299, 1.0]
3/3 [==============================] - 0s 10ms/step
pred = en gold = en
pred = es gold = es
pred = pt gold = pt
pred = en gold = en
pred = pt gold = pt
pred = ca gold = ca
pred = de gold = de
pred = ca gold = ca
pred = pt gold = pt
pred = it gold = it
pred = nl gold = nl
pred = fr gold = fr
pred = en gold = en
pred = es gold = es
pred = ca gold = ca
pred = en gold = en
pred = it gold = it
pred = nl gold = nl
pred = nl gold = nl
pred = it gold = it
pred = fr gold = fr
pred = es gold = es
pred = fr gold = fr
pred = ca gold = ca
pred = sl gold = sl
pred = sl gold = sl
pred = en gold = en
pred = pt gold = pt
pred = sl gold = sl
pred = ca gold = ca
pred = ca gold = ca
pred = es gold = es
pred = ca gold = ca
pred = de gold = de
pred = de gold = de
pred = it gold = it
pred = en gold = en
pred = sl gold = sl
pred = ca gold = ca
pred = de gold = de
pred = de gold = d

# Code pour les **unigrammes**

In [ ]:
%%bash
rm train.tgz
wget http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/train.tgz
tar xvfz train.tgz


train/
train/uk_iu-ud-train.txt
train/zh_gsd-ud-train.txt
train/la_ittb-ud-train.txt
train/af_afribooms-ud-train.txt
train/be_hse-ud-train.txt
train/cs_cac-ud-train.txt
train/fro_srcmf-ud-train.txt
train/hsb_ufal-ud-train.txt
train/mt_mudt-ud-train.txt
train/mr_ufal-ud-train.txt
train/en_lines-ud-train.txt
train/fr_sequoia-ud-train.txt
train/cu_proiel-ud-train.txt
train/fi_ftb-ud-train.txt
train/ro_rrt-ud-train.txt
train/sv_talbanken-ud-train.txt
train/hi_hdtb-ud-train.txt
train/en_esl-ud-train.txt
train/pl_sz-ud-train.txt
train/cs_pdt-ud-train.txt
train/bg_btb-ud-train.txt
train/el_gdt-ud-train.txt
train/fr_partut-ud-train.txt
train/sme_giella-ud-train.txt
train/da_ddt-ud-train.txt
train/qhe_hiencs-ud-train.txt
train/lv_lvtb-ud-train.txt
train/no_nynorsk-ud-train.txt
train/cs_fictree-ud-train.txt
train/en_ewt-ud-train.txt
train/ga_idt-ud-train.txt
train/fr_spoken-ud-train.txt
train/ca_ancora-ud-train.txt
train/sl_sst-ud-train.txt
train/ur_udtb-ud-train.txt
train/got_proiel-ud-train.tx

--2024-02-29 13:32:49--  http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/train.tgz
Resolving pageperso.lif.univ-mrs.fr (pageperso.lif.univ-mrs.fr)... 139.124.22.27
Connecting to pageperso.lif.univ-mrs.fr (pageperso.lif.univ-mrs.fr)|139.124.22.27|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pageperso.lis-lab.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/train.tgz [following]
--2024-02-29 13:32:50--  https://pageperso.lis-lab.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/train.tgz
Resolving pageperso.lis-lab.fr (pageperso.lis-lab.fr)... 139.124.22.27
Connecting to pageperso.lis-lab.fr (pageperso.lis-lab.fr)|139.124.22.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29281190 (28M) [application/x-gzip]
Saving to: ‘train.tgz’

     0K .......... .......... .......... .......... ..........  0%  221K 2m9s
    50K .......... .......... .......... .......... ..........

In [ ]:
l_corpus_train=[
['en', './train/en_partut-ud-train.txt'],
['fr', './train/fr_sequoia-ud-train.txt'],
['it', './train/it_partut-ud-train.txt'],
['nl', './train/nl_lassysmall-ud-train.txt'],
['sl', './train/sl_sst-ud-train.txt'],
['es', './train/es_ancora-ud-train.txt'],
['pt', './train/pt_bosque-ud-train.txt'],
['de', './train/de_gsd-ud-train.txt'],
['ca', './train/ca_ancora-ud-train.txt']
]

In [ ]:
def calculeCodeLangues(l_corpus):
    nbLangues = 0
    codeLangue = { }

    for corpus in l_corpus:
        idLangue = corpus[0]
        fichierCorpus = corpus[1]
        if not idLangue in codeLangue :
          print('langue :', idLangue, 'code = ', nbLangues)
          codeLangue[idLangue] = nbLangues
          nbLangues += 1
    return codeLangue

codeLangues = calculeCodeLangues(l_corpus_train)

langue : en code =  0
langue : fr code =  1
langue : it code =  2
langue : nl code =  3
langue : sl code =  4
langue : es code =  5
langue : pt code =  6
langue : de code =  7
langue : ca code =  8


In [ ]:
import random
import sys

def calculFreq(unigrammes):
    somme = 0
    for elt in unigrammes :
        somme += elt
    i = 0
    while i < len(unigrammes):
        unigrammes[i] /= somme
        i = i + 1

def afficheUnigrammes(unigrammes, fic):
    for frequence in unigrammes :
        print(frequence, ' ', file = fic, end='')
    print('', file = fic)

def char_code(c):
    if 'a' <= c and c <= 'z':
        return ord(c) - ord('a')
    elif c == ' ':
        return 26
    else :
        return 27


def unigram_code(c1):
    return char_code(c1)



def process_corpus_random(corpus, maxUnigrammes, maxTirage):
    try:
        fic = open(corpus, 'r')
    except IOError:
        print("le fichier", corpus, "n'existe pas")
        return None

    corpus_str = fic.read()
    longueur_corpus = len(corpus_str)
#    print("longueur corpus = ", longueur_corpus)
    fic.close()

    l_unigrammes = []
    tirage = 0
    while tirage < maxTirage:
        unigrammes = [0] * 28
        nbUnigrammes = 0
        while nbUnigrammes <= maxUnigrammes :
            position = random.randint(0,longueur_corpus - 1)
            code_unigramme = unigram_code(corpus_str[position])
            unigrammes[code_unigramme] += 1
            nbUnigrammes += 1
        calculFreq(unigrammes)
        l_unigrammes.append(unigrammes)
        tirage += 1
    return l_unigrammes

def extract_unigrams(l_corpus, nbUnigrammes, nbTirages, fichierSortie) :
    try:
        ficOut = open(fichierSortie, 'w')
    except IOError:
        print("le fichier", fichierSortie, "n'existe pas")
        exit

    for corpus in l_corpus:
        idLangue = corpus[0]
        fichierCorpus = corpus[1]
        print('traite corpus', fichierCorpus)
        l_unigrammes = process_corpus_random(fichierCorpus, nbUnigrammes, nbTirages)
        for unigrammes in l_unigrammes:
            print(idLangue, ' ', file = ficOut, end='')
            afficheUnigrammes(unigrammes, ficOut)
    ficOut.close()

extract_bigrams(l_corpus_train, 400, 400, 'train.dat')

traite corpus ./train/en_partut-ud-train.txt
traite corpus ./train/fr_sequoia-ud-train.txt
traite corpus ./train/it_partut-ud-train.txt
traite corpus ./train/nl_lassysmall-ud-train.txt
traite corpus ./train/sl_sst-ud-train.txt
traite corpus ./train/es_ancora-ud-train.txt
traite corpus ./train/pt_bosque-ud-train.txt
traite corpus ./train/de_gsd-ud-train.txt
traite corpus ./train/ca_ancora-ud-train.txt


In [ ]:
import numpy as np
import sys

def lectureDonnees(nomFichier, codeLangue):
    try:
        fic = open(nomFichier, 'r')
    except IOError:
        print("le fichier", nomFichier, "n'existe pas")
        return None

    nbLangues = len(codeLangue.keys())
    lx = []
    langues = []
    for ligne in fic:
        ligne = ligne.strip('\n\r')
        liste = ligne.split()
        langue = liste.pop(0)
        langues.append(codeLangue[langue])
        resultat = [float(x) for x in liste]
        lx.append(resultat)
    fic.close()

    ly = []
    for i in range(len(langues)):
        v = [0] * nbLangues
        v[langues[i]] = 1
        ly.append(v)

    import random
    t = list(zip(lx, ly))
    random.shuffle(t)
    lx, ly = list(zip(*t))

    x_train = np.array(lx, dtype="float")
    y_train = np.array(ly, dtype="int")
    print(len(x_train), "exemples lus")
    #print(x_train)
    return (x_train, y_train)



(x_train, y_train) = lectureDonnees('train.dat', codeLangues)


3600 exemples lus


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential()
nbLangues = len(codeLangues.keys())

print('nbLangues =', nbLangues)

model.add(Dense(units=100, activation='tanh', input_dim=28))
#model.add(Dropout(0.5))
model.add(Dense(units=nbLangues, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# x_train and y_train are Numpy arrays --just like in the Scikit-Learn API.
model.fit(x_train, y_train, epochs=20, batch_size=16, validation_split=0.2)

nbLangues = 9
Epoch 1/20
180/180 [==============================] - 1s 4ms/step - loss: 2.1397 - accuracy: 0.3000 - val_loss: 2.0616 - val_accuracy: 0.5208
Epoch 2/20
180/180 [==============================] - 1s 3ms/step - loss: 1.9061 - accuracy: 0.7059 - val_loss: 1.7340 - val_accuracy: 0.8806
Epoch 3/20
180/180 [==============================] - 1s 3ms/step - loss: 1.5034 - accuracy: 0.8184 - val_loss: 1.3241 - val_accuracy: 0.8986
Epoch 4/20
180/180 [==============================] - 1s 4ms/step - loss: 1.1330 - accuracy: 0.9187 - val_loss: 1.0157 - val_accuracy: 0.8792
Epoch 5/20
180/180 [==============================] - 1s 4ms/step - loss: 0.8685 - accuracy: 0.9062 - val_loss: 0.7915 - val_accuracy: 0.9139
Epoch 6/20
180/180 [==============================] - 1s 8ms/step - loss: 0.6823 - accuracy: 0.9375 - val_loss: 0.6315 - val_accuracy: 0.9250
Epoch 7/20
180/180 [==============================] - 2s 10ms/step - loss: 0.5515 - accuracy: 0.9413 - val_loss: 0.5190 - val_accuracy

In [ ]:
%%bash
rm test.tgz
wget http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/test.tgz
tar xvfz test.tgz


test/
test/cs_fictree-ud-test.txt
test/akk_pisandub-ud-test.txt
test/la_ittb-ud-test.txt
test/gl_ctg-ud-test.txt
test/bg_btb-ud-test.txt
test/sv_talbanken-ud-test.txt
test/hi_hdtb-ud-test.txt
test/fi_pud-ud-test.txt
test/et_edt-ud-test.txt
test/ga_idt-ud-test.txt
test/da_ddt-ud-test.txt
test/br_keb-ud-test.txt
test/ja_pud-ud-test.txt
test/cs_pdt-ud-test.txt
test/pl_lfg-ud-test.txt
test/no_nynorsk-ud-test.txt
test/pt_bosque-ud-test.txt
test/id_pud-ud-test.txt
test/bm_crb-ud-test.txt
test/kpv_ikdp-ud-test.txt
test/ar_padt-ud-test.txt
test/kk_ktb-ud-test.txt
test/ar_nyuad-ud-test.txt
test/hy_armtdp-ud-test.txt
test/fi_ftb-ud-test.txt
test/it_postwita-ud-test.txt
test/fa_seraji-ud-test.txt
test/en_partut-ud-test.txt
test/zh_pud-ud-test.txt
test/cs_pud-ud-test.txt
test/sk_snk-ud-test.txt
test/tr_imst-ud-test.txt
test/eu_bdt-ud-test.txt
test/got_proiel-ud-test.txt
test/ug_udt-ud-test.txt
test/kpv_lattice-ud-test.txt
test/ru_taiga-ud-test.txt
test/ru_syntagrus-ud-test.txt
test/wbp_ufal-ud-tes

--2024-02-29 13:57:32--  http://pageperso.lif.univ-mrs.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/test.tgz
Resolving pageperso.lif.univ-mrs.fr (pageperso.lif.univ-mrs.fr)... 139.124.22.27
Connecting to pageperso.lif.univ-mrs.fr (pageperso.lif.univ-mrs.fr)|139.124.22.27|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://pageperso.lis-lab.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/test.tgz [following]
--2024-02-29 13:57:33--  https://pageperso.lis-lab.fr/~alexis.nasr/Ens/MASCO_Apprentissage_Automatique/test.tgz
Resolving pageperso.lis-lab.fr (pageperso.lis-lab.fr)... 139.124.22.27
Connecting to pageperso.lis-lab.fr (pageperso.lis-lab.fr)|139.124.22.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5342409 (5.1M) [application/x-gzip]
Saving to: ‘test.tgz’

     0K .......... .......... .......... .......... ..........  0%  221K 23s
    50K .......... .......... .......... .......... ..........  1% 

In [ ]:
l_corpus_test=[
['en', './test/en_partut-ud-test.txt'],
['fr', './test/fr_sequoia-ud-test.txt'],
['it', './test/it_partut-ud-test.txt'],
['nl', './test/nl_lassysmall-ud-test.txt'],
['sl', './test/sl_sst-ud-test.txt'],
['es', './test/es_ancora-ud-test.txt'],
['pt', './test/pt_bosque-ud-test.txt'],
['de', './test/de_gsd-ud-test.txt'],
['ca', './test/ca_ancora-ud-test.txt']
]

In [ ]:
extract_trigrams(l_corpus_test, 500, 10, 'test.dat')
(x_test, y_test) = lectureDonnees('test.dat', codeLangues)

traite corpus ./test/en_partut-ud-test.txt
traite corpus ./test/fr_sequoia-ud-test.txt
traite corpus ./test/it_partut-ud-test.txt
traite corpus ./test/nl_lassysmall-ud-test.txt
traite corpus ./test/sl_sst-ud-test.txt
traite corpus ./test/es_ancora-ud-test.txt
traite corpus ./test/pt_bosque-ud-test.txt
traite corpus ./test/de_gsd-ud-test.txt
traite corpus ./test/ca_ancora-ud-test.txt
90 exemples lus


In [ ]:
l_id = [''] * len(codeLangues)
for (key, val) in (codeLangues.items()):
  #print("key =", key , 'val =', val)
  l_id[int(val)] = key

In [ ]:

def argmax(l):
  i = 1
  max = l[0]
  arg = 0
  while i < len(l) :
    if l[i] > max :
      max = l[i]
      arg = i
    i = i + 1
  return arg

score = model.evaluate(x_test, y_test)
print('test.dat score = ', score)
l_pred = model.predict(x_test, batch_size=None, verbose=1, steps=None)
i = 0
while i < len(l_pred) :
  #print(l_pred[i])
  predicted = l_id[argmax(l_pred[i])]
  gold = l_id[argmax(y_test[i])]
  print('pred =', predicted, 'gold =', gold)
  i = i + 1


3/3 [==============================] - 0s 5ms/step - loss: 0.0834 - accuracy: 0.9889
test.dat score =  [0.08341971784830093, 0.9888888597488403]
3/3 [==============================] - 0s 4ms/step
pred = sl gold = sl
pred = fr gold = fr
pred = de gold = de
pred = it gold = it
pred = pt gold = pt
pred = nl gold = nl
pred = ca gold = ca
pred = nl gold = nl
pred = it gold = it
pred = es gold = es
pred = sl gold = sl
pred = it gold = it
pred = en gold = en
pred = sl gold = sl
pred = en gold = en
pred = nl gold = nl
pred = de gold = de
pred = pt gold = pt
pred = sl gold = sl
pred = es gold = es
pred = pt gold = pt
pred = de gold = de
pred = fr gold = fr
pred = pt gold = pt
pred = de gold = de
pred = pt gold = pt
pred = de gold = de
pred = es gold = es
pred = en gold = en
pred = nl gold = nl
pred = fr gold = fr
pred = fr gold = fr
pred = nl gold = nl
pred = de gold = de
pred = en gold = en
pred = it gold = it
pred = ca gold = ca
pred = es gold = es
pred = it gold = it
pred = sl gold = sl
pred